In [1]:
import pickle
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout
from keras.activations import relu,softmax

In [2]:
x=pickle.load(open('data.pkl','rb'))
y=pickle.load(open('data_label.pkl','rb'))
output_neurons=len(set(y))

In [3]:
output_neuron=len(set(y))

In [4]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.15)

In [71]:
input_shape=x_train.shape[1]


In [64]:
def create_model(layers,activation):
    model=Sequential()
    for i,nodes in enumerate(layers):
        if i==0:
             model.add(Dense(128,input_shape=(input_shape,)))
        model.add(Activation(activation))
        model.add(Dropout(0.5))
        model.add(Dense(nodes))
        model.add(Activation(activation))
        model.add(Dropout(0.5))
    model.add(Dense(output_neuron))
    model.add(Activation('softmax'))
    model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])#sparse categorical crossentropy works on integer targets
    return model

In [65]:
from keras.wrappers.scikit_learn import KerasClassifier
model=KerasClassifier(build_fn=create_model)

C:\Users\Hp\AppData\Local\Temp\ipykernel_2932\649555917.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model=KerasClassifier(build_fn=create_model)


In [66]:
from sklearn.model_selection import GridSearchCV

In [67]:
layers=[[100],[120,90],[120,100,90]]
activations=['sigmoid','relu']
param_grid=dict(layers=layers,activation=activations,batch_size=[30,50],epochs=[100,200,75])


In [72]:
clf=GridSearchCV(estimator= model,param_grid=param_grid,cv=5,scoring='accuracy')
clf.fit(x_train,y_train,verbose=0)

1/1 [==============================] - 0s 77ms/step


GridSearchCV(cv=5,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x0000024BF0D00910>,
             param_grid={'activation': ['sigmoid', 'relu'],
                         'batch_size': [30, 50], 'epochs': [100, 200, 75],
                         'layers': [[100], [120, 90], [120, 100, 90]]},
             scoring='accuracy')

In [73]:
clf.best_params_

{'activation': 'relu', 'batch_size': 30, 'epochs': 200, 'layers': [100]}

In [74]:
best_model=create_model([100],'relu')

In [75]:
history=best_model.fit(x_train,y_train,epochs=200)

Epoch 1/200
4/4 [==============================] - 1s 5ms/step - loss: 3.1291 - accuracy: 0.0496
Epoch 2/200
4/4 [==============================] - 0s 7ms/step - loss: 3.0503 - accuracy: 0.0496
Epoch 3/200
4/4 [==============================] - 0s 6ms/step - loss: 3.0356 - accuracy: 0.1074
Epoch 4/200
4/4 [==============================] - 0s 10ms/step - loss: 2.9896 - accuracy: 0.1074
Epoch 5/200
4/4 [==============================] - 0s 6ms/step - loss: 2.9855 - accuracy: 0.1405
Epoch 6/200
4/4 [==============================] - 0s 5ms/step - loss: 2.9526 - accuracy: 0.1240
Epoch 7/200
4/4 [==============================] - 0s 6ms/step - loss: 2.8869 - accuracy: 0.1570
Epoch 8/200
4/4 [==============================] - 0s 5ms/step - loss: 2.8788 - accuracy: 0.1736
Epoch 9/200
4/4 [==============================] - 0s 4ms/step - loss: 2.8280 - accuracy: 0.1983
Epoch 10/200
4/4 [==============================] - 0s 5ms/step - loss: 2.7814 - accuracy: 0.2397
Epoch 11/200
4/4 [==========

In [76]:
best_model.evaluate(x_test,y_test)

1/1 [==============================] - 0s 144ms/step - loss: 1.2482 - accuracy: 0.7273


[1.248190999031067, 0.7272727489471436]

In [77]:
pickle.dump(best_model,open('chatbot_model.pkl','wb'))

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\activation
......vars
...layers\activation_1
......vars
...layers\activation_2
......vars
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...layers\dropout
......vars
...layers\dropout_1
......vars
...metrics\mean
......vars
.........0
.........1
...metrics\mean_metric_wrapper
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-02-10 21:01:38         2559
metadata.json                                  2023-02-10 21:01:38           64
variables.h5                                   2023-02-10 21:01: